In [ ]:
#instalamos librerias necesarias
!python -m pip install numpy pandas matplotlib yfinance statsmodels jupyter
#importamos librerias
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import yfinance as yf
import statsmodels.api as sm

#data from sqm and sp500
START = "2014-01-01"
END = "2025-12-31"
TICKER_SQM = "SQM"
TICKER_SP500 = "^GSPC"
TICKER_ETF= "LIT"

#direction of the data of litium insee monthly
LITHIUM_File = "../DATA/Processed data/lithium_insee_real.csv"
LITHIUM_SEP = ";" 



In [3]:
tickers = [TICKER_SQM, TICKER_SP500, TICKER_ETF]

#to not have problems with the dividends
prices_daily = yf.download(tickers, start=START, end=END, auto_adjust=True, progress=False)["Close"]

#if series and not dataframe, we convert it to dataframe
if isinstance(prices_daily, pd.Series):
    prices_daily = prices_daily.to_frame()

#to not have problesm with the missing data, we drop the rows with all values missing
prices_daily = prices_daily.dropna(how="all")

#conversion
prices_m = prices_daily.resample("ME").last()

prices_m= prices_m.rename(columns={TICKER_SQM: "SQM", TICKER_SP500: "SP500", TICKER_ETF: "LIT_ETF"})

#returns monthly
returns_m = prices_m.pct_change().dropna()

prices_m.tail(), returns_m.tail()


(Ticker        LIT_ETF        SQM        SP500
 Date                                         
 2025-08-31  47.331474  45.509998  6460.259766
 2025-09-30  56.696045  42.980000  6688.459961
 2025-10-31  62.151234  48.990002  6840.200195
 2025-11-30  63.288143  64.320000  6849.089844
 2025-12-31  65.519997  68.980003  6896.240234,
 Ticker       LIT_ETF       SQM     SP500
 Date                                    
 2025-08-31  0.158126  0.238367  0.019066
 2025-09-30  0.197851 -0.055592  0.035324
 2025-10-31  0.096218  0.139833  0.022687
 2025-11-30  0.018293  0.312921  0.001300
 2025-12-31  0.035265  0.072450  0.006884)

In [ ]:
#ordenamos data de litio
lith = pd.read_csv(LITHIUM_File, sep=LITHIUM_SEP)

lith["DATE"] = pd.to_datetime(lith["DATE"])
lith["PRICE"] = pd.to_numeric(lith["PRICE"], errors="coerce")

lith = lith.dropna().set_index("DATE").sort_index()

#mensual fin de mes
lith_m = lith["PRICE"].resample("ME").last().rename("lithium")

df = pd.concat([prices_m, lith_m], axis=1).dropna()
#test
df.tail()



In [ ]:
#monthly returns
rets = df.pct_change().dropna()
rets.columns
rets.tail()